Credit to @wenhuchen -  https://github.com/wenhuchen/Program-of-Thoughts

In [12]:
import json
from time import sleep
from tqdm import tqdm
import os
import openai
from datetime import datetime
from tool import finqa_equal, parse_api_result, safe_execute
from eval_tatqa.tatqa_utils import extract_one_num_from_str
from typing import Dict, Any
import argparse
from collections import Counter

In [18]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then write code to answer a question:\n'
    if example['text']:
        prompt += example['text'] + '\n'
    prompt += example['table'].strip() + '\n'
    prompt += 'Question: {}\n'.format(example['question'])
    return prompt

In [19]:
finqa_prompt = [{'role': 'system',
  'content': 'You are an AI assistant that helps people find information.'},
 {'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
( in millions ) | dec 282013 | dec 292012
available-for-sale investments | $ 18086 | $ 14001
cash | 854 | 593
equity method investments | 1038 | 992
loans receivable | 1072 | 979
non-marketable cost method investments | 1270 | 1202
reverse repurchase agreements | 800 | 2850
trading assets | 8441 | 5685
total cash and investments | $ 31561 | $ 26302
Question: what percentage of total cash and investments as of dec . 29 2012 was comprised of available-for-sale investments?'''},
 {'role': 'assistant',
  'content': '''#Python
available_for_sale_investments_dec_29_2012 = 14001
total_cash_and_investments_dec_29_2012 = 26302
percent_available_for_sale_investments_dec_29_2012 = available_for_sale_investments_dec_29_2012 / total_cash_and_investments_dec_29_2012
ans = percent_available_for_sale_investments_dec_29_2012'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
the chart shows that the firm posted market risk 2013related gains on 248 out of 261 days in this period , with 12 days exceeding $ 210 million .
december 31 ( in millions ) | 1 basis point increase in jpmorgan chase 2019s credit spread
2010 | $ 35
2009 | $ 39
Question: on what percent of trading days were there market gains above $ 210 million?'''},
{'role': 'assistant',
  'content': '''#Python
days_with_market_gains_above_210_million = 12
total_trading_days = 261
percent_days_with_market_gains_above_210_million = days_with_market_gains_above_210_million / total_trading_days
ans = percent_days_with_market_gains_above_210_million'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
american tower corporation and subsidiaries notes to consolidated financial statements ( 3 ) consists of customer-related intangibles of approximately $ 75.0 million and network location intangibles of approximately $ 72.7 million . the customer-related intangibles and network location intangibles are being amortized on a straight-line basis over periods of up to 20 years .
- | preliminary purchase price allocation
current assets | $ 8763
non-current assets | 2332
property and equipment | 26711
intangible assets ( 1 ) | 21079
other non-current liabilities | -1349 ( 1349 )
fair value of net assets acquired | $ 57536
goodwill ( 2 ) | 5998
Question: for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?'''},
{'role': 'assistant',
  'content': '''#Python
customer_related_intangibles = 75
network_location_intangibles = 72.7
straight_line_basis = 20
amortization_expenses = ( customer_related_intangibles + network_location_intangibles ) / straight_line_basis
ans = amortization_expenses'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
the aggregate commitment under the liquidity asset purchase agreements was approximately $ 23.59 billion and $ 28.37 billion at december 31 , 2008 and 2007 , respectively .
( dollars in billions ) | 2008 amount | 2008 percent of total conduit assets | 2008 amount | percent of total conduit assets
united states | $ 11.09 | 46% ( 46 % ) | $ 12.14 | 42% ( 42 % )
australia | 4.30 | 17 | 6.10 | 21
great britain | 1.97 | 8 | 2.93 | 10
spain | 1.71 | 7 | 1.90 | 7
italy | 1.66 | 7 | 1.86 | 7
portugal | 0.62 | 3 | 0.70 | 2
germany | 0.57 | 3 | 0.70 | 2
netherlands | 0.40 | 2 | 0.55 | 2
belgium | 0.29 | 1 | 0.31 | 1
greece | 0.27 | 1 | 0.31 | 1
other | 1.01 | 5 | 1.26 | 5
total conduit assets | $ 23.89 | 100% ( 100 % ) | $ 28.76 | 100% ( 100 % )
Question: what is percentage change in total conduit asset from 2007 to 2008?'''},
{'role': 'assistant',
  'content': '''#Python
total_conduit_assets_2007 = 28.76
total_conduit_assets_2008 = 23.89
net_change_in_total_conduit_assets = total_conduit_assets_2008 - total_conduit_assets_2007
percent_change_in_total_conduit_assets = net_change_in_total_conduit_assets / total_conduit_assets_2007
ans = percent_change_in_total_conduit_assets'''},]

In [20]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2023-03-15-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  'gpt-35-turbo' #parser.get('openai_api','api_model')

os.environ["OPENAI_API_KEY"]  = openai.api_key
os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_VERSION"] = openai.api_version
os.environ["OPENAI_API_BASE"] = openai.api_base


In [21]:
with open('data/finqa_test.json') as f:
        finqa_dev = json.load(f)
    
now = datetime.now()
dt_string = now.strftime("%m_%d_%H_%M")

correct, wrong = 0, 0

In [22]:
filename = f'outputs/finqa_{model}_{dt_string}.jsonl'
writer = open(filename, 'w')

for example in tqdm(finqa_dev):
    full_prompt = finqa_prompt + [{"role":"user","content":create_reader_request_processed(example)}]
    # greedy decoding
    got_result = False
    while not got_result:
        try:
            result = openai.ChatCompletion.create(
                engine=model,
                messages=full_prompt,
                max_tokens=256,
                temperature=0.0,
                top_p=1,
                n=1,
                stop=['\n\n']
            )
            got_result = True
        except Exception:
            sleep(3)
    result_counter = Counter()
    codes = result['choices'][0]['message']['content'],

    for r in codes:
        try :
            exec(r)#extract_one_num_from_str(r)
        except:
            ans=r
        if not ans:
            if 'yes' in r.lower() or 'true' in r.lower():
                ans = 'yes'
            elif 'no' in r.lower() or 'false' in r.lower():
                ans = 'no'
        if ans is not None:
            if type(ans) in [dict]:
                result_counter.update(list(ans.values()))
            elif type(ans) in [list, tuple]:
                result_counter.update([float(ans[0])])
            elif type(ans) in [str]:
                result_counter.update([ans])
            else:
                try:
                    result_counter.update([float(ans)])
                except Exception:
                    continue

    if len(result_counter) > 0:
        prediction = result_counter.most_common(1)[0][0]        
    else:
        prediction = None

    if prediction is None:
        wrong += 1
    elif finqa_equal(prediction, example['answer'], True, True):
        correct += 1
    else:
        wrong += 1

    example.update({'generated': codes, 'executed': prediction})
    writer.write(json.dumps(example) + '\n')
    print('accuracy: ', correct / (correct + wrong))
    
writer.close()


  0%|                                                                                 | 1/1147 [00:02<53:40,  2.81s/it]

accuracy:  1.0


  0%|▏                                                                                | 2/1147 [00:04<44:01,  2.31s/it]

accuracy:  1.0


  0%|▏                                                                                | 3/1147 [00:06<34:57,  1.83s/it]

...

accuracy:  0.611353711790393


100%|█████████████████████████████████████████████████████████████████████████████▉| 1146/1147 [53:17<00:02,  2.10s/it]

accuracy:  0.6116928446771379


100%|██████████████████████████████████████████████████████████████████████████████| 1147/1147 [53:20<00:00,  2.79s/it]

accuracy:  0.6120313862249346


In [24]:
print('accuracy: ', correct / (correct + wrong))


accuracy:  0.6120313862249346
